In [24]:
%pip install imutils cmake dlib face_recognition numpy pandas scikit-learn opencv-contrib-python-headless pillow firebase_admin google-cloud-storage google-cloud-bigquery google-cloud-aiplatform google-cloud-pipeline-components kfp logger

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for logger: filename=logger-1.4-py3-none-any.whl size=1761 sha256=4d83407eb9af8f07cf5539095ee16e01157d10292ae4b02280c0789d74051b6d
  Stored in directory: c:\users\parth\appdata\local\pip\cache\wheels\cd\1d\c8\f1361043ff09fd3c9a747b199e8e37bd716bacf0843bbdb68f
Successfully built logger
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\mask_rcnn-2.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [11]:
# Importing necessary modules

import os
from google.cloud import aiplatform

from kfp.dsl import pipeline, component
from kfp import compiler

In [12]:
PROJECT_ID = "plenary-truck-411220"
PIPELINE_ROOT = "gs://faces_for_clusters"

In [13]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../keys/detectionKey.json"

In [14]:
aiplatform.init(project=PROJECT_ID, location="asia-south1")

In [15]:
@component(base_image="python:3.10", packages_to_install=["numpy==1.23.5", "opencv-contrib-python-headless==4.9.0.80", "imutils==0.5.4", "db-dtypes==1.2.0", "scikit-learn==1.4.1.post1", "pillow==10.2.0", "build==1.0.3", "cmake==3.28.3", "dlib==19.24.2", "face-recognition==1.3.0", "logger==1.4", "requests==2.31.0", "pandas==2.2.0", "google-cloud-storage==2.14.0", "google-cloud-bigquery==3.17.2", "kfp==2.6.0"])
def fetch_cropped_imgs_and_clustering()->bool:
    
    from google.cloud import bigquery, storage
    import cv2
    import numpy as np
    from PIL import Image
    import dlib
    import os
    import logging
    import traceback
    import requests
    from imutils import paths, build_montages
    import face_recognition
    from pandas import DataFrame
    from sklearn.cluster import DBSCAN

    logging.basicConfig(level=logging.DEBUG)

    logger = logging.getLogger(__name__)

    def move_image(image: cv2.typing.MatLike, id: int, labelID: int) -> None:
        """
        Move the image to a labeled directory.

        Parameters:
        - image (cv2.typing.MatLike): The image data.
        - id (int): The identifier for the image.
        - labelID (int): The label identifier for the image.

        Returns:
        - None
        """
        try:
            path = os.getcwd() + '/label' + str(labelID)

            if os.path.exists(path) == False:
                os.mkdir(path)

            filename = str(id) + '.jpg'

            cv2.imwrite(os.path.join(path, filename), image)
        except Exception as e:
            logger.critical(f"An error occurred: {str(e)}")


    def load_img(path:str) -> cv2.typing.MatLike:
        img = cv2.imread(path)
        return img


    def draw_predict(
        frame:cv2.typing.MatLike, 
        left:int, 
        top:int, 
        right:int, 
        bottom:int
        )->None:
        cv2.rectangle(
            frame, 
            (left, top), 
            (right, bottom), 
            (0, 0, 255), 
            3
        )


    def get_eyes_nose_dlib(shape):
        nose = shape[4][1]
        left_eye_x = int(shape[3][1][0] + shape[2][1][0]) // 2
        left_eye_y = int(shape[3][1][1] + shape[2][1][1]) // 2
        right_eyes_x = int(shape[1][1][0] + shape[0][1][0]) // 2
        right_eyes_y = int(shape[1][1][1] + shape[0][1][1]) // 2
        return nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)


    def get_eyes_nose(eyes, nose):
        left_eye_x = int(eyes[0][0] + eyes[0][2] / 2)
        left_eye_y = int(eyes[0][1] + eyes[0][3] / 2)
        right_eye_x = int(eyes[1][0] + eyes[1][2] / 2)
        right_eye_y = int(eyes[1][1] + eyes[1][3] / 2)
        nose_x = int(nose[0][0] + nose[0][2] / 2)
        nose_y = int(nose[0][1] + nose[0][3] / 2)

        return (nose_x, nose_y), (right_eye_x, right_eye_y), (left_eye_x, left_eye_y)


    def rotate_point(origin, point, angle):
        ox, oy = origin
        px, py = point

        qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
        qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
        return qx, qy


    def is_between(point1, point2, point3, extra_point):
        c1 = (point2[0] - point1[0]) * (extra_point[1] - point1[1]) - (point2[1] - point1[1]) * (extra_point[0] - point1[0])
        c2 = (point3[0] - point2[0]) * (extra_point[1] - point2[1]) - (point3[1] - point2[1]) * (extra_point[0] - point2[0])
        c3 = (point1[0] - point3[0]) * (extra_point[1] - point3[1]) - (point1[1] - point3[1]) * (extra_point[0] - point3[0])
        if (c1 < 0 and c2 < 0 and c3 < 0) or (c1 > 0 and c2 > 0 and c3 > 0):
            return True
        else:
            return False


    def distance(a, b):
        return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)


    def cosine_formula(length_line1, length_line2, length_line3):
        cos_a = -(length_line3 ** 2 - length_line2 ** 2 - length_line1 ** 2) / (2 * length_line2 * length_line1)
        return cos_a


    def show_img(img):
        while True:
            cv2.imshow('face_alignment_app', img)
            c = cv2.waitKey(1)
            if c == 27:
                break
        cv2.destroyAllWindows()


    def shape_to_normal(shape):
        shape_normal = []
        for i in range(0, 5):
            shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
        return shape_normal


    def rotate_opencv(img, nose_center, angle):
        M = cv2.getRotationMatrix2D(nose_center, angle, 1)
        rotated = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_CUBIC)
        return rotated


    def rotation_detection_dlib(img, mode, show=False):
        detector = dlib.get_frontal_face_detector()#type:ignore
        predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')#type:ignore
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)
        if len(rects) > 0:
            for rect in rects:
                x = rect.left()
                y = rect.top()
                w = rect.right()
                h = rect.bottom()
                shape = predictor(gray, rect)
                shape = shape_to_normal(shape)
                nose, left_eye, right_eye = get_eyes_nose_dlib(shape)
                center_of_forehead = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
                center_pred = (int((x + w) / 2), int((y + y) / 2))
                length_line1 = distance(center_of_forehead, nose)
                length_line2 = distance(center_pred, nose)
                length_line3 = distance(center_pred, center_of_forehead)
                cos_a = cosine_formula(length_line1, length_line2, length_line3)
                angle = np.arccos(cos_a)
                rotated_point = rotate_point(nose, center_of_forehead, angle)
                rotated_point = (int(rotated_point[0]), int(rotated_point[1]))
                if is_between(nose, center_of_forehead, center_pred, rotated_point):
                    angle = np.degrees(-angle)
                else:
                    angle = np.degrees(angle)

                if mode:
                    img = rotate_opencv(img, nose, angle)
                else:
                    img = Image.fromarray(img)
                    img = np.array(img.rotate(angle))
            if show:
                show_img(img)
            return img
        else:
            return img


    def rotation_detection_opencv(img, mode, show=False):
        nose_cascade = cv2.CascadeClassifier('haarcascade_mcs_nose.xml')
        eyes_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
        fase_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        nose_rects = nose_cascade.detectMultiScale(gray, 1.3, 5)
        eyes_rects = eyes_cascade.detectMultiScale(gray, 1.3, 5)
        face_rects = fase_cascade.detectMultiScale(gray, 1.3, 5)
        length_eyes = len(eyes_rects)

        if length_eyes == 2 and len(nose_rects) != 0 and len(face_rects) != 0:
            nose, right_eye, left_eye = get_eyes_nose(eyes_rects, nose_rects)
        else:
            print("Couldn't determine eyes/nose")
            return img
        center_of_forehead = (int((right_eye[0] + left_eye[0]) / 2), int((right_eye[1] + left_eye[1]) / 2))
        center_pred = (int((face_rects[0][0] + face_rects[0][2]) / 2), int((face_rects[0][1] + face_rects[0][1]) / 2))
        length_line1 = distance(center_of_forehead, nose)
        length_line2 = distance(center_pred, nose)
        length_line3 = distance(center_pred, center_of_forehead)
        cos_a = cosine_formula(length_line1, length_line2, length_line3)
        angle = np.arccos(cos_a)
        rotated_point = rotate_point(nose, center_of_forehead, angle)
        rotated_point = (int(rotated_point[0]), int(rotated_point[1]))
        if is_between(nose, center_of_forehead, center_pred, rotated_point):
            angle = np.degrees(-angle)
        else:
            angle = np.degrees(angle)
        if mode:
            img = rotate_opencv(img, nose, angle)
        else:
            img = Image.fromarray(img)
            img = np.array(img.rotate(angle))
        if show:
            show_img(img)
        return img


    def save_img(path, img):
        cv2.imwrite(path, img)


    def face_alignment(
        path_to_load:str,
        mode:int = 1
        )->cv2.typing.MatLike:
        """Align FAces

        Args:
            path_to_load (str): path to load.
            mode (int, optional): 0 for Cv2
                                1 for dlib. Defaults to 1.

        Returns:
            cv2.typing.MatLike: _description_
        """
        img = load_img(path_to_load)
        if mode == 0:
            img = rotation_detection_opencv(img, 0)
        else:
            img = rotation_detection_dlib(img, 0, False)
        return img


    class FaceCluster:

        def __init__(
            self,
            jobs: int = 1,
            metric: str = "euclidean",
            # save: bool = False,
            show: bool = False,
        ) -> None:
            """
            Initialize the FaceCluster object.

            Parameters:
            - jobs (int): Number of parallel jobs to run for DBSCAN (default is 1).
            - metric (str): Metric used for clustering (default is 'euclidean').
            - save (bool): Whether to save the clustered images (default is False).
            - show (bool): Whether to display the clustered faces montage (default is False).

            Returns:
            - None
            """
            self.metric = metric
            self.clt = DBSCAN(metric=self.metric, n_jobs=jobs)
            # self.save = save
            self.show = show

        def rotate_opencv(
            self, img: cv2.typing.MatLike, nose_center: tuple, angle: float
        ) -> cv2.typing.MatLike:
            """
            Rotate an image using OpenCV.

            Parameters:
            - img (cv2.typing.MatLike): Input image.
            - nose_center (tuple): Coordinates of the nose center.
            - angle (float): Rotation angle.

            Returns:
            - cv2.typing.MatLike: Rotated image.
            """
            M = cv2.getRotationMatrix2D(nose_center, angle, 1)
            rotated = cv2.warpAffine(
                img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_CUBIC
            )
            return rotated

        def is_between(
            self, point1: tuple, point2: tuple, point3: tuple, extra_point: tuple
        ) -> bool:
            """
            Check if extra_point lies between point1, point2, and point3.

            Parameters:
            - point1 (tuple): Coordinates of the first point.
            - point2 (tuple): Coordinates of the second point.
            - point3 (tuple): Coordinates of the third point.
            - extra_point (tuple): Coordinates of the extra point.

            Returns:
            - bool: True if extra_point is between the three points, False otherwise.
            """
            c1 = (point2[0] - point1[0]) * (extra_point[1] - point1[1]) - (
                point2[1] - point1[1]
            ) * (extra_point[0] - point1[0])
            c2 = (point3[0] - point2[0]) * (extra_point[1] - point2[1]) - (
                point3[1] - point2[1]
            ) * (extra_point[0] - point2[0])
            c3 = (point1[0] - point3[0]) * (extra_point[1] - point3[1]) - (
                point1[1] - point3[1]
            ) * (extra_point[0] - point3[0])
            if (c1 < 0 and c2 < 0 and c3 < 0) or (c1 > 0 and c2 > 0 and c3 > 0):
                return True
            else:
                return False

        def rotate_point(self, origin: tuple, point: tuple, angle: float) -> tuple:
            """
            Rotate a point around an origin by a specified angle.

            Parameters:
            - origin (tuple): Coordinates of the origin.
            - point (tuple): Coordinates of the point to be rotated.
            - angle (float): Rotation angle.

            Returns:
            - tuple: Coordinates of the rotated point.
            """
            ox, oy = origin
            px, py = point

            qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
            qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
            return qx, qy

        def cosine_formula(
            self, length_line1: float, length_line2: float, length_line3: float
        ) -> float:
            """
            Compute the cosine of an angle using the cosine formula.

            Parameters:
            - length_line1 (float): Length of the first line.
            - length_line2 (float): Length of the second line.
            - length_line3 (float): Length of the third line.

            Returns:
            - float: Cosine of the angle.
            """
            cos_a = -(length_line3**2 - length_line2**2 - length_line1**2) / (
                2 * length_line2 * length_line1
            )
            return cos_a

        def distance(self, a: tuple, b: tuple) -> float:
            """
            Calculate the Euclidean distance between two points.

            Parameters:
            - a (tuple): Coordinates of the first point.
            - b (tuple): Coordinates of the second point.

            Returns:
            - float: Euclidean distance between the two points.
            """
            return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

        def get_eyes_nose_dlib(self, shape: dlib.full_object_detection) -> tuple:  # type: ignore
            """
            Extract coordinates of nose, left eye, and right eye from dlib shape object.

            Parameters:
            - shape (dlib.full_object_detection): Detected facial landmarks.

            Returns:
            - tuple: Tuple containing coordinates of nose, left eye, and right eye.
            """
            nose = shape[4][1]
            left_eye_x = int(shape[3][1][0] + shape[2][1][0]) // 2
            left_eye_y = int(shape[3][1][1] + shape[2][1][1]) // 2
            right_eyes_x = int(shape[1][1][0] + shape[0][1][0]) // 2
            right_eyes_y = int(shape[1][1][1] + shape[0][1][1]) // 2

            return nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)

        def shape_to_normal(self, shape: dlib.full_object_detection) -> list:  # type: ignore
            """
            Convert dlib shape object to a list of tuples.

            Parameters:
            - shape (dlib.full_object_detection): Detected facial landmarks.

            Returns:
            - list: List of tuples representing coordinates of facial landmarks.
            """
            shape_normal = []
            for i in range(0, 5):
                shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
            return shape_normal

        def rotation_detection_dlib(
            self, img: cv2.typing.MatLike, mode: int, show: bool = False
        ) -> cv2.typing.MatLike:
            """
            Perform rotation detection using dlib's facial landmarks.

            Parameters:
            - img (cv2.typing.MatLike): Input image.
            - mode (int): Rotation mode (0: no rotation, 1: rotate).
            - show (bool): Whether to display the rotated image (default is False).

            Returns:
            - cv2
            """
            detector = dlib.get_frontal_face_detector()  # type: ignore
            predictor = dlib.shape_predictor("shape_predictor_5_face_landmarks.dat")  # type: ignore
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            rects = detector(gray, 0)
            if len(rects) > 0:
                for rect in rects:
                    x = rect.left()
                    y = rect.top()
                    w = rect.right()
                    h = rect.bottom()
                    shape = predictor(gray, rect)
                    shape = self.shape_to_normal(shape)
                    nose, left_eye, right_eye = self.get_eyes_nose_dlib(shape)
                    center_of_forehead = (
                        (left_eye[0] + right_eye[0]) // 2,
                        (left_eye[1] + right_eye[1]) // 2,
                    )
                    center_pred = (int((x + w) / 2), int((y + h) / 2))
                    length_line1 = self.distance(center_of_forehead, nose)
                    length_line2 = self.distance(center_pred, nose)
                    length_line3 = self.distance(center_pred, center_of_forehead)
                    cos_a = self.cosine_formula(length_line1, length_line2, length_line3)
                    angle = np.arccos(cos_a)
                    rotated_point = self.rotate_point(nose, center_of_forehead, angle)
                    rotated_point = (int(rotated_point[0]), int(rotated_point[1]))
                    if self.is_between(
                        nose, center_of_forehead, center_pred, rotated_point
                    ):
                        angle = np.degrees(-angle)
                    else:
                        angle = np.degrees(angle)

                    if mode:
                        img = self.rotate_opencv(img, nose, angle)
                    else:
                        img = Image.fromarray(img)  # type: ignore
                        img = np.array(img.rotate(angle))  # type: ignore
                if show:
                    show_img(img)
                return img
            else:
                return img

        def face_align(
            self, imagePath: str, mode: int = 0, show: bool = False
        ) -> cv2.typing.MatLike:
            """
            Perform face alignment using rotation detection.

            Parameters:
            - imagePath (str): Path to the input image.
            - mode (int): Rotation mode (0: no rotation, 1: rotate).
            - show (bool): Whether to display the aligned image (default is False).

            Returns:
            - cv2.typing.MatLike: Aligned face image.
            """
            img = load_img(imagePath)
            img_rot = self.rotation_detection_dlib(img, mode, show)
            return img_rot

        def face_align_df(
            self, imagePath: str, mode: int = 0, show: bool = False
        ) -> cv2.typing.MatLike | None:
            """
            Perform face alignment using rotation detection.

            Parameters:
            - imagePath (str): Path to the input image.
            - mode (int): Rotation mode (0: no rotation, 1: rotate).
            - show (bool): Whether to display the aligned image (default is False).

            Returns:
            - cv2.typing.MatLike: Aligned face image.
            """
            try:
                response = requests.get(imagePath)

                if response.status_code == 200:
                    image_array = np.frombuffer(response.content, dtype=np.uint8)
                    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
                    # img = cv2.imread(imagePath)
                    img_rot = self.rotation_detection_dlib(image, mode, show)
                    return img_rot
                else:
                    logger.debug(
                        "Failed to fetch the image. Status code:", response.status_code
                    )
            except Exception as e:
                logger.debug("An error occurred:", str(e))

        def getEncodings(self, dataset: str, detection_method: str = "cnn") -> list | None:
            """
            Extract facial encodings from a dataset of images.

            Parameters:
            - dataset (str): Path to the dataset directory.
            - detection_method (str): Face detection method (default is "cnn").

            Returns:
            - list: List of dictionaries containing image information and encodings.
            """
            logger.info("Quantifying faces...")
            try:
                imagePaths = list(paths.list_images(dataset))
                logger.debug(f"Images loaded: {len(imagePaths)}")

                data = []

                for i, imagePath in enumerate(imagePaths):

                    # aligned_img_path = os.path.join(os.getcwd(),"Temp",os.path.basename(imagePath))
                    # logger.debug(aligned_img_path)
                    logger.debug("Processing image {}/{}".format(i + 1, len(imagePaths)))
                    logger.debug(imagePath)

                    image_ip = self.face_align(imagePath)
                    # cv2.imwrite(aligned_img_path, image_ip)
                    image_ = cv2.cvtColor(image_ip, cv2.COLOR_BGR2GRAY)
                    image = cv2.cvtColor(image_, cv2.COLOR_GRAY2BGR)

                    boxes = face_recognition.face_locations(image, model=detection_method)

                    encodings = face_recognition.face_encodings(image, boxes)

                    d = [
                        {
                            # "coorectedImagePath":aligned_img_path,
                            "imagePath": imagePath,
                            "id": os.path.basename(imagePath)[:-4],
                            "loc": box,
                            "encoding": enc,
                        }
                        for (box, enc) in zip(boxes, encodings)
                    ]

                    data.extend(d)

                logger.debug("serializing encodings...")
                return data  # type: ignore

            except Exception as e:
                logger.critical(f"An error occurred: {str(e)}")

        def getEncodingsFromDf(
            self, dataset: DataFrame, detection_method: str = "cnn"
        ) -> list | None:
            """
            Extract facial encodings from a dataset of images.

            Parameters:
            - dataset (df): Path to the dataset directory.
            - detection_method (str): Face detection method (default is "cnn").

            Returns:
            - list: List of dictionaries containing image information and encodings.
            """
            logger.info("Quantifying faces...")
            try:
                imagePaths = dataset["gcsUrl"].to_list()
                logger.debug(f"Images loaded: {len(imagePaths)}")

                data = []

                for i, imagePath in enumerate(imagePaths):
                    logger.debug("Processing image {}/{}".format(i + 1, len(imagePaths)))
                    logger.debug(imagePath)

                    image = self.face_align_df(imagePath)
                    try:
                        boxes = face_recognition.face_locations(image, model=detection_method)

                        encodings = face_recognition.face_encodings(image, boxes)

                        d = [
                            {
                                "imagePath": imagePath,
                                "id": os.path.basename(imagePath)[:-4],
                                "loc": box,
                                "encoding": enc,
                            }
                            for (box, enc) in zip(boxes, encodings)
                        ]

                        data.extend(d)
                    except Exception as e:
                        logger.critical(f"An error occurred: {str(e)}")
                        continue

                logger.debug("serializing encodings...")
                return data  # type: ignore

            except Exception as e:
                logger.critical(f"An error occurred: {str(e)}")

        def getClusters(self, dataset: str | DataFrame) -> dict | None:
            """
            Perform face clustering on a dataset and return the clustered results.

            Parameters:
            - dataset (str): Path to the dataset directory.

            Returns:
            - dict: Dictionary containing clustered face IDs.
            """
            logger.debug(type(dataset))
            if type(dataset) == str:
                self.encodings = self.getEncodings(dataset=dataset)
                logger.info("Received local filesystem dataset.")
            elif type(dataset) == DataFrame:
                self.encodings = self.getEncodingsFromDf(dataset=dataset)
                logger.info("Received df dataset.")
            try:
                logger.info("Loading encodings...")
                data = self.encodings
                encodings = [d["encoding"] for d in data]  # type:ignore

                logger.info("Clustering ...")

                self.clt.fit(encodings)  # type: ignore

                labelIDs = np.unique(self.clt.labels_)

                self.numUniqueFaces = len(np.where(labelIDs > -1)[0])
                logger.info("# unique faces: {}".format(self.numUniqueFaces))

                res_dict = {}

                for labelID in labelIDs:
                    logger.info("Faces for face ID: {}".format(labelID))
                    idxs = np.where(self.clt.labels_ == labelID)[0]
                    idxs = np.random.choice(idxs, size=min(25, len(idxs)), replace=False)
                    logger.debug(self.clt.labels_)

                    faces = []

                    ids = []
                    for i in idxs:
                        # image = cv2.imread(data[i]["imagePath"])  # type:ignore

                        ids.append(data[i]["id"])  # type:ignore
                        # top, right, bottom, left = data[i]["loc"]  # type:ignore
                        # face = image[top:bottom, left:right]

                        # if self.save:
                        #     move_image(image, i, labelID)

                        # face = cv2.resize(face, (96, 96))
                        # faces.append(face)

                    res_dict[str(labelID)] = ids

                    # if self.save:
                    #     montage = build_montages(faces, (96, 96), (5, 5))[0]
                    #     title = "Face ID #{}".format(labelID)
                    #     title = "Unknown Faces" if labelID == -1 else title

                    #     if self.show:
                    #         cv2.imshow(title, montage)
                    #         cv2.waitKey(0)
                    #     cv2.imwrite(os.path.join(os.getcwd(), title + ".jpg"), montage)

                return res_dict

            except Exception as e:
                traceback_str = traceback.format_exc()
                logger.critical(f"An error occurred: {str(e)}. {traceback_str}")
        
    storage_client = storage.Client("plenary-truck-411220")

    bucket_pipeline = storage_client.get_bucket("faces_for_clustersdetection_pipeline")
    # Create a blob object from the filepath
    blob = bucket_pipeline.blob("keys/detectionKey.json")
    face_landmarks = bucket_pipeline.blob("shape_predictor_5_face_landmarks.dat")
    # Download the file to a destination
    blob.download_to_filename("detectionKey.json")
    face_landmarks.download_to_filename("shape_predictor_5_face_landmarks.dat")
    CLOUD_CRED = "detectionKey.json"
    client = bigquery.Client.from_service_account_json(CLOUD_CRED)

    sql_query = """
        SELECT * FROM `plenary-truck-411220.clustering_dataset.faces`
        """
    df = client.query(sql_query).result().to_dataframe()
    data = FaceCluster()
    res = data.getClusters(dataset=df)

    res_exists = res != None
    if res_exists:
        for cluster_id in res.keys():
            if (cluster_id != '-1'):
                update_query = f"""
                    UPDATE `plenary-truck-411220.clustering_dataset.faces` SET cluster={int(cluster_id)}
                    WHERE markedId IN {tuple(res[cluster_id])}
                """
                client.query(update_query)   

    return res_exists

In [16]:
@component(base_image="python:3.10", packages_to_install=["numpy==1.23.5", "opencv-contrib-python-headless==4.9.0.80", "imutils==0.5.4", "db-dtypes==1.2.0", "pillow==10.2.0", "build==1.0.3", "cmake==3.28.3", "dlib==19.24.2", "face-recognition==1.3.0", "logger==1.4", "requests==2.31.0", "pandas==2.2.0", "google-cloud-storage==2.14.0", "google-cloud-bigquery==3.17.2", "kfp==2.6.0"])
def classify_images(prev_task: bool):
    from google.cloud import storage, bigquery
    import face_recognition
    import os
    from PIL import Image
    import requests
    import logging
    import tempfile
    import numpy as np
    import pandas as pd

    logging.basicConfig(level=logging.DEBUG)

    logger = logging.getLogger(__name__)

    if prev_task:
        storage_client = storage.Client("plenary-truck-411220")

        bucket_pipeline = storage_client.get_bucket("faces_for_clustersdetection_pipeline")
        # Create a blob object from the filepath
        blob = bucket_pipeline.blob("keys/detectionKey.json")
        # Download the file to a destination
        blob.download_to_filename("detectionKey.json")
        CLOUD_CRED = "detectionKey.json"
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = CLOUD_CRED
        sql_query = """
            SELECT * FROM `plenary-truck-411220.clustering_dataset.faces`
        """
        client = bigquery.Client()
        df = client.query(sql_query).result().to_dataframe().fillna(-1)
        unique_clusters = df['cluster'].unique().tolist()
        clusters = {str(key): [] for key in unique_clusters}

        with tempfile.TemporaryDirectory() as tempDir:
            logger.debug("Iterating through the table . . .")
            for _, row in df.iterrows():
                img = Image.open(requests.get(row['gcsUrl'], stream=True).raw)
                id = row["markedId"]
                img.save(os.path.join(tempDir, f"{id}.png"))
                img_path = os.path.join(tempDir, f"{id}.png")
                clusters[str(row['cluster'])].append([id, img_path])
                known_encodings = {str(key): [] for key in unique_clusters}
                for key in clusters.keys():
                    if key != "-1":
                        for imgs in clusters[key]:
                            known_img = face_recognition.load_image_file(imgs[1])
                            encodings = face_recognition.face_encodings(known_img, num_jitters=2)[0]
                            known_encodings[key].append(encodings)
                        logger.debug(f"Encoding created for {key} . . .")
                
                for uncl in clusters["-1"]:
                    unknown_img = face_recognition.load_image_file(uncl[1])
                    encoding_uk = face_recognition.face_encodings(unknown_img)

                    logger.debug(f"Checking for IMG {uncl[0]}")

                    if len(encoding_uk) > 0:
                        encoding_uk = encoding_uk[0]
                    else:
                        continue

                    for key in known_encodings.keys():
                        res = face_recognition.compare_faces(known_encodings[key], encoding_uk, tolerance=0.4)
                        logger.info(res)
                        if any(res):
                            client.query(f"UPDATE `plenary-truck-411220.clustering_dataset.faces` SET cluster={int(key)} WHERE markedId={uncl[0]}")
                            logger.debug(f"Added to CLUSTER {key}")

In [17]:
@pipeline(name="clustering_classification_pipeline", pipeline_root=PIPELINE_ROOT + "detection_pipeline")
def detection_pipeline():
    clustering_task = (fetch_cropped_imgs_and_clustering()).set_memory_limit('64G')
    classification_task = classify_images(prev_task = clustering_task.output)

In [18]:
compiler.Compiler().compile(
    pipeline_func=detection_pipeline, package_path="clus_class_pipeline.yaml" # type: ignore
)

In [19]:
job = aiplatform.PipelineJob(
    display_name="clustering_classification_pipeline",
    template_path="./clus_class_pipeline.yaml"
)

In [ ]:
job.submit(
    service_account="detection@plenary-truck-411220.iam.gserviceaccount.com"
)